# 1) Cargar la data

In [1]:
import os, pickle
import pandas as pd



def encontrar_activos_y_temporalidades(elementos_SQ, temporalidades):
    """
    Encuentra los elementos anteriores al primer match entre la lista de elementos_SQ y temporalidades.

    Args:
        elementos_SQ (list): Lista de elementos a verificar.
        temporalidades (list): Lista de temporalidades a buscar.

    Returns:
        list: Lista de listas que contienen los elementos anteriores al primer match para cada elemento en elementos_SQ.
        list: Lista de listas que contienen las temporalidades encontradas en cada elemento de elementos_SQ.
    """
    resultados = []
    temporalidades_encontradas = []
    
    for elemento in elementos_SQ:
        elementos_previos = []
        
        for temporalidad in temporalidades:
            index = elemento.find(temporalidad)
            if index != -1:
                elementos_previos.extend(elemento[:index].split('_')[:-1])
                temporalidades_encontradas.append(elemento[index:].split('_')[:-1])
                break
        
        resultados.append(elementos_previos)
    
    return resultados, temporalidades_encontradas

# Load the data
date_path = os.path.join(os.getcwd(), 'data')

# Get the date directory
print(f' Directorios guardados: {os.listdir(date_path)}')

file_dir = os.path.join(date_path, os.listdir(date_path)[-1])#TODO: estamos eligiendo el ultimo directorio, HARDCODED
print(f' Cantidad de solicitudes guardadas: {len(os.listdir(file_dir))//4}')

# obtener los nombres de los activos descargados
environment_temporalities = ['1d','4h','2h','1h','30m','15m','5m','3m', '1m']
elementos_SQ = [file for file in os.listdir(file_dir) if file.endswith('SQ.pickle')]

activos, temporalidades = encontrar_activos_y_temporalidades(elementos_SQ, environment_temporalities)


if __name__ == '__main__':
    activos_elegidos = activos[0]
    temporalidades_correspondientes = temporalidades[0]
    archivos_necesarios = ['.pickle','_SQ.pickle', '_CORR.pickle', '_TW.pickle']
    file_name_without_extension = '_'.join(activos_elegidos)+'_'+'_'.join(temporalidades_correspondientes)
    loaded_files = []
    for extension in archivos_necesarios:
        file_name = file_name_without_extension + extension
        file_path = os.path.join(file_dir, file_name)
        with open(file_path, 'rb') as file:
            loaded_files.append(pickle.load(file))
    data = {
        'data_raw': loaded_files[0],
        'data_SQ': loaded_files[1],
        'SQ_CORR': loaded_files[2],
        'time_window': loaded_files[3]
    }

 Directorios guardados: ['04_junio_2023', '05_junio_2023', '06_junio_2023', '07_junio_2023']
 Cantidad de solicitudes guardadas: 2


In [2]:
# Output
data.keys()
data['data_raw'].keys()
data['data_raw']['BTC'].keys()
data['data_raw']['BTC']['1d']


data['data_SQ'].keys()
data['data_SQ'][22]['BTC'].keys()
data['data_SQ'][22]['BTC']['1d']


data['SQ_CORR'].keys()
data['SQ_CORR'][22].keys()
data['SQ_CORR'][22]['BTC']['1d']


data['time_window'].keys()
data['time_window']['1d'].keys()
data['time_window']['1d'][22]


'2023-02-07 13:59:53'

# 2) Mostrar las correlaciones

Podría utilizarse la libreria Rich para armar tablas y exportarlas en formato txt


In [4]:
from tabulate import tabulate


def get_correlation(data, activo):#TODO: llevar a un modulo
    # Filter the data dictionary based on the activo
    filtered_data = {
        key: values[activo.upper()]
        for key, values in data.items()
    }

    # Create the DataFrame
    df = pd.DataFrame(filtered_data)
    df.columns.name = 'correlacion {}'.format(activo)

    return df


if __name__ == '__main__':
    asset = 'XRP'

    df = get_correlation(data['SQ_CORR'], asset)

    # tabla para exportar a txt
    tabla = tabulate(df, headers='keys', tablefmt='fancy_grid')
    print(f'Correlación {asset} con Squeeze Momentum Indicator')
    print(tabla)

    # formato para la impresion con estilo
    keys= list(df.columns)
    values = ['{:.2f}' for i in range(len(keys))]
    format_dict = dict(zip(keys, values))

df.style.format(format_dict).background_gradient(cmap='PRGn')

Correlación XRP con Squeeze Momentum Indicator
╒═════╤═══════╤══════╤═══════╤══════╤══════╤═══════╤═══════╤══════╤══════╤═══════╤══════╤══════╤══════╕
│     │     9 │   12 │    14 │   18 │   20 │    22 │    30 │   40 │   50 │    60 │   70 │   80 │   90 │
╞═════╪═══════╪══════╪═══════╪══════╪══════╪═══════╪═══════╪══════╪══════╪═══════╪══════╪══════╪══════╡
│ 1d  │  0.8  │ 0.81 │  0.46 │ 0.52 │ 0.56 │  0.59 │  0.62 │ 0.59 │ 0.53 │  0.31 │ 0.3  │ 0.45 │ 0.58 │
├─────┼───────┼──────┼───────┼──────┼──────┼───────┼───────┼──────┼──────┼───────┼──────┼──────┼──────┤
│ 4h  │  0.77 │ 0.64 │ -0.13 │ 0.17 │ 0.2  │  0.23 │  0.34 │ 0.51 │ 0.57 │  0.56 │ 0.57 │ 0.6  │ 0.62 │
├─────┼───────┼──────┼───────┼──────┼──────┼───────┼───────┼──────┼──────┼───────┼──────┼──────┼──────┤
│ 2h  │  0.61 │ 0.77 │  0.75 │ 0.2  │ 0.18 │  0.17 │  0.09 │ 0.09 │ 0.19 │  0.56 │ 0.62 │ 0.68 │ 0.73 │
├─────┼───────┼──────┼───────┼──────┼──────┼───────┼───────┼──────┼──────┼───────┼──────┼──────┼──────┤
│ 1h  │ -0.01 │ 0

correlacion XRP,9,12,14,18,20,22,30,40,50,60,70,80,90
1d,0.80,0.81,0.46,0.52,0.56,0.59,0.62,0.59,0.53,0.31,0.30,0.45,0.58
4h,0.77,0.64,-0.13,0.17,0.20,0.23,0.34,0.51,0.57,0.56,0.57,0.60,0.62
2h,0.61,0.77,0.75,0.20,0.18,0.17,0.09,0.09,0.19,0.56,0.62,0.68,0.73
1h,-0.01,0.21,0.46,0.61,0.67,0.72,0.80,0.30,0.13,0.11,0.17,0.27,0.36
30m,0.78,0.53,-0.15,0.48,0.51,0.54,0.67,0.68,0.78,0.22,0.23,0.21,0.16
15m,0.66,0.71,0.79,0.10,0.00,-0.08,-0.10,0.51,0.59,0.42,0.53,0.63,0.72
5m,0.67,0.78,0.78,0.68,0.67,0.67,0.70,0.25,0.13,0.19,0.18,0.19,0.23
3m,0.43,0.54,0.65,0.73,0.76,0.77,0.76,0.67,0.70,-0.07,0.04,0.16,0.24
1m,0.34,0.20,0.27,0.28,0.32,0.36,0.47,0.63,0.71,0.63,0.68,0.71,0.72


In [4]:
# Una forma auxiliar de imprimir dataframes
from rich import print

print(f'Correlación {asset} con Squeeze Momentum Indicator')
print(df.loc[:,70:])
print(df.loc[:,30:60])
print(df.loc[:,9:22])


Correlación BTC con Squeeze Momentum Indicator

correlacion BTC    70    80    90
1d               0.15  0.16  0.28
4h               0.41  0.19  0.04
2h               0.29  0.19  0.20
1h               0.67  0.66  0.65
30m              0.53  0.57  0.62
15m              0.52  0.57  0.61
5m               0.21  0.26  0.30
3m               0.01  0.17  0.28
1m               0.50  0.56  0.60

correlacion BTC    30    40    50    60
1d               0.28  0.46  0.54  0.20
4h               0.56  0.32  0.31  0.53
2h               0.67  0.64  0.62  0.40
1h               0.67  0.47  0.58  0.65
30m              0.70  0.58  0.65  0.49
15m              0.38  0.53  0.60  0.45
5m               0.68  0.23  0.18  0.18
3m               0.64  0.67  0.74 -0.13
1m               0.25  0.27  0.33  0.44

correlacion BTC    9     12    14    18    20    22
1d               0.52  0.55  0.29  0.34  0.33  0.32
4h               0.64  0.64  0.61  0.65  0.66  0.65
2h               0.53  0.65  0.61  0.51  0.54  0.57
1h               0.52  0.72  0.45  0.53  0.58  0.61
30m              0.44 -0.12  0.32  0.50  0.53  0.56
15m              0.73  0.77  0.62  0.20  0.19  0.20
5m               0.29  0.42  0.55  0.57  0.60  0.63
3m               0.19  0.28  0.27  0.47  0.50  0.53
1m               0.64  0.53  0.41  0.11  0.14  0.17

# 3) Crear un registro de las notas de las lecturas

In [ ]:
# Revisar tradegestor DEMO

# 4) Visualización de la data

# 5) Exportar tablas, registros y visualizaciones